In [1]:
import os
import shutil
import csv
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt 
from IPython.display import clear_output
from torch.utils.data import Dataset, DataLoader, random_split
from scipy.signal import butter, filtfilt, find_peaks
from torch import linalg as LA

In [2]:
## Low Pass Filter for Azure data
def butter_lowpass_filter(data, cutoff, order):
    normal_cutoff=cutoff/(15) 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    x = np.expand_dims(filtfilt(b, a, data[:,0]), axis=-1)
    y = np.expand_dims(filtfilt(b, a, data[:,1]), axis=-1)
    z = np.expand_dims(filtfilt(b, a, data[:,2]), axis=-1)
    filtered = np.concatenate((x, y, z), axis=-1)
    return filtered

In [3]:
def ReadCSV(path):
    csv_data = np.loadtxt(path, delimiter=',', skiprows=1, usecols=range(1, 101))
    
    # LPF를 위한 변수
    cutoff = 6
    order = 30
    
    pelvis_idx = 0
    
    LShoulder_idx = 5
    LElbow_idx = 6
    LWrist_idx = 7
    LHip_idx = 18 
    LKnee_idx = 19
    LAnkle_idx = 20
    
    RShoulder_idx = 12
    RElbow_idx = 13
    RWrist_idx = 14
    RHip_idx = 22
    RKnee_idx = 23
    RAnkle_idx = 24
    
    Pelvis = csv_data[:,pelvis_idx * 3:(pelvis_idx + 1) * 3]
    Pelvis_filtered = butter_lowpass_filter(Pelvis, cutoff, order)
    
    LShoulder = csv_data[:,LShoulder_idx * 3:(LShoulder_idx + 1) * 3]
    LShoulder_filtered = butter_lowpass_filter(LShoulder, cutoff, order)
    
    LElbow = csv_data[:,LElbow_idx * 3:(LElbow_idx + 1) * 3]
    LElbow_filtered = butter_lowpass_filter(LElbow, cutoff, order)
    
    LWrist = csv_data[:,LWrist_idx * 3:(LWrist_idx + 1) * 3]
    LWrist_filtered = butter_lowpass_filter(LWrist, cutoff, order)
    
    LHip = csv_data[:,LHip_idx * 3:(LHip_idx + 1) * 3]
    LHip_filtered = butter_lowpass_filter(LHip, cutoff, order)
    
    LKnee = csv_data[:,LKnee_idx * 3:(LKnee_idx + 1) * 3]
    LKnee_filtered = butter_lowpass_filter(LKnee, cutoff, order)
    
    LAnkle = csv_data[:,LAnkle_idx * 3:(LAnkle_idx + 1) * 3]
    LAnkle_filtered = butter_lowpass_filter(LAnkle, cutoff, order)
    
    RShoulder = csv_data[:,RShoulder_idx * 3:(RShoulder_idx + 1) * 3]
    RShoulder_filtered = butter_lowpass_filter(RShoulder, cutoff, order)
    
    RElbow = csv_data[:,RElbow_idx * 3:(RElbow_idx + 1) * 3]
    RElbow_filtered = butter_lowpass_filter(RElbow, cutoff, order)
    
    RWrist = csv_data[:,RWrist_idx * 3:(RWrist_idx + 1) * 3]
    RWrist_filtered = butter_lowpass_filter(RWrist, cutoff, order)
    
    RHip = csv_data[:,RHip_idx * 3:(RHip_idx + 1) * 3]
    RHip_filtered = butter_lowpass_filter(RHip, cutoff, order)
    
    RKnee = csv_data[:,RKnee_idx * 3:(RKnee_idx + 1) * 3]
    RKnee_filtered = butter_lowpass_filter(RKnee, cutoff, order)
    
    RAnkle = csv_data[:,RAnkle_idx * 3:(RAnkle_idx + 1) * 3]
    RAnkle_filtered = butter_lowpass_filter(RAnkle, cutoff, order)
    

    Pelvis_vel = Pelvis_filtered[1:,:] - Pelvis_filtered[:-1,:]
    LElbow_vel = LElbow_filtered[1:,:] - LElbow_filtered[:-1,:]
    LKnee_vel = LKnee_filtered[1:,:] - LKnee_filtered[:-1,:]
    RElbow_vel = RElbow_filtered[1:,:] - RElbow_filtered[:-1,:]
    RKnee_vel = RKnee_filtered[1:,:] - RKnee_filtered[:-1,:]
    
    ## Pelvis에 대한 무릎과 팔꿈치의 상대적인 움직임
    Pelvis_vel_nomalized = Pelvis_vel / np.linalg.norm(Pelvis_vel, axis=1, keepdims=True)
    
    LElbow_alongPelvis_norm = np.expand_dims(np.tensordot(Pelvis_vel_nomalized, LElbow_vel, axes=([1],[1])).diagonal(), axis=-1)    
    LElbow_alongPelvis = Pelvis_vel_nomalized * LElbow_alongPelvis_norm
    LElbow_vertical2Pelvis = LElbow_vel - LElbow_alongPelvis
    LElbow_vertical2Pelvis_norm = np.linalg.norm(LElbow_vertical2Pelvis, axis=1, keepdims=True)
    
    LKnee_alongPelvis_norm = np.expand_dims(np.tensordot(Pelvis_vel_nomalized, LKnee_vel, axes=([1],[1])).diagonal(), axis=-1)    
    LKnee_alongPelvis = Pelvis_vel_nomalized * LKnee_alongPelvis_norm
    LKnee_vertical2Pelvis = LKnee_vel - LKnee_alongPelvis
    LKnee_vertical2Pelvis_norm = np.linalg.norm(LKnee_vertical2Pelvis, axis=1, keepdims=True)
    
    RElbow_alongPelvis_norm = np.expand_dims(np.tensordot(Pelvis_vel_nomalized, RElbow_vel, axes=([1],[1])).diagonal(), axis=-1)    
    RElbow_alongPelvis = Pelvis_vel_nomalized * RElbow_alongPelvis_norm
    RElbow_vertical2Pelvis = RElbow_vel - RElbow_alongPelvis
    RElbow_vertical2Pelvis_norm = np.linalg.norm(RElbow_vertical2Pelvis, axis=1, keepdims=True)
    
    RKnee_alongPelvis_norm = np.expand_dims(np.tensordot(Pelvis_vel_nomalized, RKnee_vel, axes=([1],[1])).diagonal(), axis=-1)    
    RKnee_alongPelvis = Pelvis_vel_nomalized * RKnee_alongPelvis_norm
    RKnee_vertical2Pelvis = RKnee_vel - RKnee_alongPelvis
    RKnee_vertical2Pelvis_norm = np.linalg.norm(RKnee_vertical2Pelvis, axis=1, keepdims=True)
    
    
    ## 무릎,팔꿈치 각도, 0번째 데이터는 삭제해야함. 왜냐하면 위의 velocity 가 1번부터 시작
    knee2hip = LHip_filtered - LKnee_filtered
    knee2hip = knee2hip / np.linalg.norm(knee2hip, axis = 1, keepdims=True)
    knee2angkle = LAnkle_filtered - LKnee_filtered
    knee2angkle = knee2angkle / np.linalg.norm(knee2angkle, axis = 1, keepdims=True)
    inner = np.sum(np.multiply(knee2hip, knee2angkle), axis=-1)
    LKneeAngle = np.expand_dims(180 - np.arccos(inner) * 180 / np.pi, axis=-1)[1:,:]
    
    elbow2wrist = LWrist_filtered - LElbow_filtered
    elbow2wrist = elbow2wrist / np.linalg.norm(elbow2wrist, axis = 1, keepdims=True)
    elbow2shoulder = LShoulder_filtered - LElbow_filtered
    elbow2shoulder = elbow2shoulder / np.linalg.norm(elbow2shoulder, axis = 1, keepdims=True)
    inner = np.sum(np.multiply(elbow2wrist, elbow2shoulder), axis=-1)
    LElbowAngle = np.expand_dims(180 - np.arccos(inner) * 180 / np.pi, axis=-1)[1:,:]
    
    knee2hip = RHip_filtered - RKnee_filtered
    knee2hip = knee2hip / np.linalg.norm(knee2hip, axis = 1, keepdims=True)
    knee2angkle = RAnkle_filtered - RKnee_filtered
    knee2angkle = knee2angkle / np.linalg.norm(knee2angkle, axis = 1, keepdims=True)
    inner = np.sum(np.multiply(knee2hip, knee2angkle), axis=-1)
    RKneeAngle = np.expand_dims(180 - np.arccos(inner) * 180 / np.pi, axis=-1)[1:,:]
    
    elbow2wrist = RWrist_filtered - RElbow_filtered
    elbow2wrist = elbow2wrist / np.linalg.norm(elbow2wrist, axis = 1, keepdims=True)
    elbow2shoulder = RShoulder_filtered - RElbow_filtered
    elbow2shoulder = elbow2shoulder / np.linalg.norm(elbow2shoulder, axis = 1, keepdims=True)
    inner = np.sum(np.multiply(elbow2wrist, elbow2shoulder), axis=-1)
    RElbowAngle = np.expand_dims(180 - np.arccos(inner) * 180 / np.pi, axis=-1)[1:,:]
    
    data = np.concatenate((LKneeAngle, LElbowAngle, RKneeAngle, RElbowAngle, LElbow_alongPelvis_norm, LElbow_vertical2Pelvis_norm, \
                           LKnee_alongPelvis_norm, LKnee_vertical2Pelvis_norm, RElbow_alongPelvis_norm, RElbow_vertical2Pelvis_norm, \
                          RKnee_alongPelvis_norm, RKnee_vertical2Pelvis_norm), axis=-1)  
    
    return data
    

In [4]:
def ReadLabel(path):
    dat = open(path) 
    reader = csv.reader(dat) 
    line = list(reader)[-1]
    answer = []
    for i in line:
        answer.append(int(i.split('_')[-1].split('.')[0]) - 1)
    return answer

In [5]:
class CustomDataset(Dataset):
    def __init__(self, root):
        
        global inputDims
        
        self.root = root
        
        interval = [15, 0] # 두 수의 합이 인풋의 길이. 앞의 수는 과거부터 현재까지의 프레임 수, 뒤의 수는 미래의 프레임 수 
        self.single_inputs = None
        self.single_outputs = None
        
        initBuffer = False
        
        for name in next(os.walk(self.root))[1]:
            #print(name)
            for trial in next(os.walk(self.root + '//' + name))[1]:
                trialPath = self.root + '//' + name + '//' + trial
                
                csvPath = trialPath + '//' + 'skeleton_' + name + '_' + trial + '.csv'
                labelPath = trialPath + '//' + 'label_' + name + '_' + trial + '.csv'
                
                
                try:
                    data = ReadCSV(csvPath)
                    label = ReadLabel(labelPath)
                
                    # Label
                    # 0 : sit
                    # 1 : walking
                    # 2 : Turning
                    # exceptFrame 액션 사이의 경계를 기준으로 모호한 영역을 지정하고 제외시키기 위한 변수
                    exceptFrame = 10

                    # startMove까지 sit
                    targetFrame = interval[0] - 1
                    while (targetFrame + interval[1] + 1) <  (label[0] - exceptFrame):
                        single_input = np.expand_dims(data[targetFrame - interval[0] + 1:targetFrame + interval[1] + 1, :], axis = 0)
                        single_output = np.expand_dims(np.array([0]), axis=0)

                        if not initBuffer:
                            self.single_inputs = single_input
                            self.single_outputs = single_output
                            initBuffer = True
                        else:
                            self.single_inputs = np.concatenate((self.single_inputs, single_input), axis = 0)
                            self.single_outputs = np.concatenate((self.single_outputs, single_output), axis = 0)
                        targetFrame += 1

                    # startWalk 부터 startTurn 까지 walking
                    targetFrame = label[1] + exceptFrame
                    while (targetFrame + interval[1] + 1) < (label[2] - exceptFrame):
                        single_input = np.expand_dims(data[targetFrame - interval[0] + 1:targetFrame + interval[1] + 1, :], axis = 0)
                        single_output = np.expand_dims(np.array([1]), axis=0)

                        self.single_inputs = np.concatenate((self.single_inputs, single_input), axis = 0)
                        self.single_outputs = np.concatenate((self.single_outputs, single_output), axis = 0)

                        targetFrame += 1

                    # startTurn 부터 endTurn 까지 turing
                    targetFrame = label[2] + exceptFrame
                    while (targetFrame + interval[1] + 1) < (label[3] - exceptFrame):
                        single_input = np.expand_dims(data[targetFrame - interval[0] + 1:targetFrame + interval[1] + 1, :], axis = 0)
                        single_output = np.expand_dims(np.array([2]), axis=0)

                        self.single_inputs = np.concatenate((self.single_inputs, single_input), axis = 0)
                        self.single_outputs = np.concatenate((self.single_outputs, single_output), axis = 0)

                        targetFrame += 1

                    # endTurn 부터 startSit 까지 walking
                    targetFrame = label[3] + exceptFrame
                    while (targetFrame + interval[1] + 1) < (label[4] - exceptFrame):
                        single_input = np.expand_dims(data[targetFrame - interval[0] + 1:targetFrame + interval[1] + 1, :], axis = 0)
                        single_output = np.expand_dims(np.array([1]), axis=0)

                        self.single_inputs = np.concatenate((self.single_inputs, single_input), axis = 0)
                        self.single_outputs = np.concatenate((self.single_outputs, single_output), axis = 0)

                        targetFrame += 1

                    # endSit 이후 sit
                    targetFrame = label[5] + exceptFrame
                    while (targetFrame + interval[1] + 1) < (data.shape[0]):
                        single_input = np.expand_dims(data[targetFrame - interval[0] + 1:targetFrame + interval[1] + 1, :], axis = 0)
                        single_output = np.expand_dims(np.array([0]), axis=0)

                        self.single_inputs = np.concatenate((self.single_inputs, single_input), axis = 0)
                        self.single_outputs = np.concatenate((self.single_outputs, single_output), axis = 0)

                        targetFrame += 1
                        
                except:
                    print(csvPath)
                    
        inputDims = self.single_inputs.shape[-1]
        print('for single inputs')           
        print(self.single_inputs.shape)
        print(self.single_outputs.shape)
        
        
        ## combined
        initBuffer = False
        self.combined_inputs = None
        self.combined_outputs = None
        
        self.combinedNum = 9000
        self.classNum = 3
        self.classIdx = []
        
        for i in range(self.classNum):
            idx = np.where(self.single_outputs == i)[0]
            self.classIdx.append(idx)
        
        # for same class combination
        for i in range(self.classNum):
            numberForclass = int(self.combinedNum / self.classNum)
            choosen1_idx = np.random.choice(self.classIdx[i], size=numberForclass, replace=False)
            choosen2_idx = np.random.choice(self.classIdx[i], size=numberForclass, replace=False)
            
            choosen1 = np.expand_dims(self.single_inputs[choosen1_idx], axis=1)
            choosen2 = np.expand_dims(self.single_inputs[choosen2_idx], axis=1)
            
            combined = np.concatenate((choosen1, choosen2), axis = 1)
            
            if not initBuffer:
                self.combined_inputs = combined
                self.combined_outputs = np.full((numberForclass, 1), 0)
            else:
                self.combined_inputs = np.concatenate((self.combined_inputs, combined), axis = 0)
                self.combined_outputs = np.concatenate((self.combined_outputs, np.full((numberForclass, 1), 0)), axis = 0)
            
        
        
            initBuffer = True
        
        # for diff class combination
        for i in range(self.combinedNum):
            choosedClasses = np.random.choice(np.arange(self.classNum), size = 2, replace = False)
            choosen1_idx = np.random.choice(self.classIdx[choosedClasses[0]], size=1, replace=False)
            choosen2_idx = np.random.choice(self.classIdx[choosedClasses[1]], size=1, replace=False)
            
            choosen1 = np.expand_dims(self.single_inputs[choosen1_idx], axis=1)
            choosen2 = np.expand_dims(self.single_inputs[choosen2_idx], axis=1)
            
            combined = np.concatenate((choosen1, choosen2), axis = 1)
            
            self.combined_inputs = np.concatenate((self.combined_inputs, combined), axis = 0)
            self.combined_outputs = np.concatenate((self.combined_outputs, np.full((1, 1), 1)), axis = 0)
            
        # shuffle
        s = np.arange(self.combined_inputs.shape[0])
        np.random.shuffle(s)
        self.combined_inputs = self.combined_inputs[s]
        self.combined_outputs = self.combined_outputs[s]
        
        print('for combined inputs')
        print(self.combined_inputs.shape)
        print(self.combined_outputs.shape)
            
    def __len__(self):
        return len(self.combined_inputs)
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.combined_inputs[idx,:,:])
        y = torch.FloatTensor([self.combined_outputs[idx]])
        return x, y

In [6]:
dataset = CustomDataset('TUG_dataset//HMM_saveResults_illness_mean//0_sideView//elder')

dataset_size = len(dataset)
train_size = int(0.7 * dataset_size)
test_size = dataset_size - train_size
train_dataset, test_dataset = random_split(dataset,[train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

TUG_dataset//HMM_saveResults_illness_mean//0_sideView//elder//JSM_TUG//01//skeleton_JSM_TUG_01.csv
TUG_dataset//HMM_saveResults_illness_mean//0_sideView//elder//JSM_TUG//05//skeleton_JSM_TUG_05.csv
TUG_dataset//HMM_saveResults_illness_mean//0_sideView//elder//JSM_TUG//15//skeleton_JSM_TUG_15.csv
TUG_dataset//HMM_saveResults_illness_mean//0_sideView//elder//JSM_TUG//08//skeleton_JSM_TUG_08.csv
TUG_dataset//HMM_saveResults_illness_mean//0_sideView//elder//JSM_TUG//14//skeleton_JSM_TUG_14.csv
TUG_dataset//HMM_saveResults_illness_mean//0_sideView//elder//JSM_TUG//06//skeleton_JSM_TUG_06.csv
TUG_dataset//HMM_saveResults_illness_mean//0_sideView//elder//JSM_TUG//10//skeleton_JSM_TUG_10.csv
TUG_dataset//HMM_saveResults_illness_mean//0_sideView//elder//JSM_TUG//02//skeleton_JSM_TUG_02.csv
TUG_dataset//HMM_saveResults_illness_mean//0_sideView//elder//JSM_TUG//03//skeleton_JSM_TUG_03.csv
TUG_dataset//HMM_saveResults_illness_mean//0_sideView//elder//JSM_TUG//09//skeleton_JSM_TUG_09.csv
TUG_datase

In [16]:
# 학습에 사용할 CPU나 GPU 장치를 얻습니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
#device = "cpu"

class tcnPhase(nn.Module):
    def __init__(self):
        super(tcnPhase, self).__init__()
        global hidden_size
        self.conv1 = nn.Conv1d(in_channels=inputDims, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(3, 3, 3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(3, 3, 3, stride=1, padding=1)
        
        self.input_size = 3
        self.hidden_size = 3
        hidden_size = self.hidden_size
        self.num_layers = 1 
        self.bidirectional = False
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional)
        
        if self.bidirectional:
            self.fc = nn.Linear(self.hidden_size * 2, 3)
        else:
            self.fc = nn.Linear(self.hidden_size, 3)
            
        self.Boundary=nn.Parameter(torch.randn(1))
        self.relu = nn.ReLU()
            
    def forward(self, x, h0, c0):
        x = torch.transpose(x, 1, 2)
        x = self.conv1(x)
        x = torch.sigmoid(x)
        x = self.conv2(x)
        #x = torch.sigmoid(x)
        x = self.conv3(x)
        #x = torch.sigmoid(x)
        x = torch.transpose(x, 1, 2)
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = out[:,-1,:]
        x = self.fc(out)
        x = torch.sigmoid(x)
        return x
    
    def predict(self, p1, p2):
        distance = LA.norm(p1 - p2, dim=1, keepdim = True) 
        Dinominator = distance + self.relu(self.Boundary.repeat(distance.shape)) + torch.full(distance.shape,  1e-10).to(device)
        predFinal = distance / Dinominator
        return predFinal

Using cuda device


In [17]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train(True)
    for batch, (X, y) in enumerate(dataloader):
        input1 = X[:,0,:,:].to(device)
        input2 = X[:,1,:,:].to(device)
        output = y[:,0,:].to(device)
        
        h0 = torch.zeros(1, input1.size(0), hidden_size).to(device)
        c0 = torch.zeros(1, input1.size(0), hidden_size).to(device)
        
        pred1 = model(input1, h0.detach(), c0.detach())
        pred2 = model(input2, h0.detach(), c0.detach())
        
        predFinal = model.predict(pred1, pred2)
        loss = loss_fn(predFinal, output)
        loss.backward()
        
        optimizer.step()
        
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            input1 = X[:,0,:,:].to(device)
            input2 = X[:,1,:,:].to(device)
            output = y[:,0,:].to(device)

            h0 = torch.zeros(1, input1.size(0), hidden_size).to(device)
            c0 = torch.zeros(1, input1.size(0), hidden_size).to(device)

            pred1 = model(input1, h0.detach(), c0.detach()).to(device)
            pred2 = model(input2, h0.detach(), c0.detach()).to(device)
            
            predFinal = model.predict(pred1, pred2)
            test_loss += loss_fn(predFinal, output).item()
            
            predFinal = predFinal > 0.5
            correct += (output == predFinal).sum().item()
            #correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(model.Boundary)
    #print(f"Avg loss: {test_loss:>8f} \n")
    return test_loss

In [ ]:
model = tcnPhase().to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


EPOCHS = 100000
patience = 100
stopped_epoch = 0
best = np.Inf
wait = 0
PATH = 'Mymodel_TUG'

lossHistory = []
    
for t in range(EPOCHS):
    if t%10 == 0:
        clear_output(wait=True)
    print(f"Epoch {t+1}\n-------------------------------")
    
    train(train_dataloader, model, loss_fn, optimizer)
    testLoss = test(test_dataloader, model, loss_fn)
    lossHistory.append(testLoss)
    
    if testLoss < best:
        best = testLoss
        torch.save(model.state_dict(), PATH)
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            stopped_epoch = t
            device = torch.device("cuda")
            model.load_state_dict(torch.load(PATH, map_location="cuda:0"))
            model.to(device)
            break
print("Done!")

plt.plot(lossHistory)
#plt.title('InputType' + str(inputType))
plt.tight_layout()
#plt.savefig('InputType' + str(inputType) + '_' + root + '_LossGraph')
#plt.close()

Epoch 81
-------------------------------
Test Error: 
 Accuracy: 50.1%, Avg loss: 49.981509 

Parameter containing:
tensor([-1.6805], device='cuda:0', requires_grad=True)
Epoch 82
-------------------------------
